In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [4]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2020-01-02 21:00:00,1.31385,1.31395,1.31270,1.31280,1.31280,1.31385,8.000000,92.000000,84.000000,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02 22:00:00,1.31279,1.31446,1.31277,1.31376,1.31279,1.31376,1.183432,58.579882,57.396450,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-02 23:00:00,1.31377,1.31416,1.31340,1.31376,1.31376,1.31377,47.368421,48.684211,1.315789,...,84.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-03 00:00:00,1.31358,1.31460,1.31302,1.31312,1.31312,1.31358,6.329114,35.443038,29.113924,...,57.396450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-03 01:00:00,1.31308,1.31470,1.31297,1.31458,1.31308,1.31458,6.358382,93.063584,86.705202,...,1.315789,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [5]:
df_an = BollingerBandsFeature(df_an)
df_an = ATRFeature(df_an)
df_an = KeltnerChannelsFeature(df_an)
df_an = RSIFeature(df_an)
df_an = MACDFeature(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,Feat_gains,Feat_wins_rma,Feat_losses_rma,Feat_RSI_14,MACD,SIGNAL_MACD,HIST,Feat_MACD,Feat_SIGNAL_MACD,Feat_HIST
0,2020-01-02 21:00:00,1.31385,1.31395,1.31270,1.31280,1.31280,1.31385,8.000000,92.000000,84.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02 22:00:00,1.31279,1.31446,1.31277,1.31376,1.31279,1.31376,1.183432,58.579882,57.396450,...,0.00096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02 23:00:00,1.31377,1.31416,1.31340,1.31376,1.31376,1.31377,47.368421,48.684211,1.315789,...,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-03 00:00:00,1.31358,1.31460,1.31302,1.31312,1.31312,1.31358,6.329114,35.443038,29.113924,...,-0.00064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-03 01:00:00,1.31308,1.31470,1.31297,1.31458,1.31308,1.31458,6.358382,93.063584,86.705202,...,0.00146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,2023-03-23 14:00:00,1.23026,1.23375,1.22650,1.22752,1.22752,1.23026,14.068966,51.862069,37.793103,...,-0.00274,0.000583,0.000665,0.466905,0.001185,0.001575,-0.000390,0.001185,0.001575,-0.000390
19995,2023-03-23 15:00:00,1.22753,1.23184,1.22732,1.23148,1.22753,1.23148,4.646018,92.035398,87.389381,...,0.00396,0.000824,0.000618,0.571516,0.001226,0.001505,-0.000279,0.001226,0.001505,-0.000279
19996,2023-03-23 16:00:00,1.23148,1.23187,1.22950,1.23144,1.23144,1.23148,81.856540,83.544304,1.687764,...,-0.00004,0.000765,0.000576,0.570299,0.001241,0.001452,-0.000211,0.001241,0.001452,-0.000211
19997,2023-03-23 17:00:00,1.23146,1.23304,1.23105,1.23205,1.23146,1.23205,20.603015,50.251256,29.648241,...,0.00061,0.000754,0.000535,0.584823,0.001287,0.001419,-0.000132,0.001287,0.001419,-0.000132


In [6]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'BB_MA', 'BB_UP',
       'BB_LW', 'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c', 'Feat_BB_MA_o',
       'Feat_BB_UP_o', 'Feat_BB_LW_o', 'Feat_BB_MA_l', 'Feat_BB_UP_l',
       'Feat_BB_LW_l', 'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
       'Feat_ATR_14', 'EMA', 'KeUp', 'KeLo', 'Feat_EMA_c', 'Feat_KeUp_c',
       'Feat_KeLo_c', 'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 'Feat_EMA_h',
       'Feat_KeUp_h', 'Feat_KeLo_h', 'Feat_EMA_l', 'Feat_KeUp_l',
       'Feat_KeLo_l', 'Feat_gains

In [7]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

9e-05

In [8]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,HIST,Feat_MACD,Feat_SIGNAL_MACD,Feat_HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2020-01-06 06:00:00,1.30808,1.30859,1.30798,1.30831,1.30808,1.30831,16.393443,54.098361,37.704918,...,0.000175,-0.000445,-0.000620,0.000175,False,False,1.30798,1.30894,1.30788,1.30850
1,2020-01-06 07:00:00,1.30830,1.30850,1.30788,1.30799,1.30799,1.30830,17.741935,67.741935,50.000000,...,0.000162,-0.000412,-0.000573,0.000162,False,False,1.30798,1.30859,1.30676,1.30824
2,2020-01-06 08:00:00,1.30800,1.30824,1.30676,1.30817,1.30800,1.30817,83.783784,95.270270,11.486486,...,0.000161,-0.000368,-0.000528,0.000161,True,False,1.30788,1.30850,1.30691,1.30935
3,2020-01-06 09:00:00,1.30817,1.30935,1.30691,1.30727,1.30727,1.30817,14.754098,51.639344,36.885246,...,0.000103,-0.000397,-0.000500,0.000103,False,False,1.30676,1.30824,1.30631,1.31072
4,2020-01-06 10:00:00,1.30727,1.31072,1.30631,1.31048,1.30727,1.31048,21.768707,94.557823,72.789116,...,0.000259,-0.000172,-0.000431,0.000259,True,False,1.30691,1.30935,1.31018,1.31475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,2023-03-23 13:00:00,1.22886,1.23070,1.22822,1.23026,1.22886,1.23026,25.806452,82.258065,56.451613,...,-0.000154,0.001519,0.001672,-0.000154,False,False,1.22841,1.23162,1.22650,1.23375
19958,2023-03-23 14:00:00,1.23026,1.23375,1.22650,1.22752,1.22752,1.23026,14.068966,51.862069,37.793103,...,-0.000390,0.001185,0.001575,-0.000390,True,True,1.22822,1.23070,1.22732,1.23184
19959,2023-03-23 15:00:00,1.22753,1.23184,1.22732,1.23148,1.22753,1.23148,4.646018,92.035398,87.389381,...,-0.000279,0.001226,0.001505,-0.000279,False,False,1.22650,1.23375,1.22950,1.23187
19960,2023-03-23 16:00:00,1.23148,1.23187,1.22950,1.23144,1.23144,1.23148,81.856540,83.544304,1.687764,...,-0.000211,0.001241,0.001452,-0.000211,False,False,1.22732,1.23184,1.23105,1.23304


In [80]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
SPREAD_IN_POSITON = 0# SPREAD # 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] - SPREAD_IN_POSITON
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] - SPREAD_IN_POSITON) + df_an.close[i]
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] + SPREAD_IN_POSITON
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] - SPREAD_IN_POSITON) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_13672\942741458.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13672\942741458.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13672\942741458.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13672\942741458.py:26: SettingWithCopyWarning:


A v

In [65]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [66]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'BB_MA', 'BB_UP',
       'BB_LW', 'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c', 'Feat_BB_MA_o',
       'Feat_BB_UP_o', 'Feat_BB_LW_o', 'Feat_BB_MA_l', 'Feat_BB_UP_l',
       'Feat_BB_LW_l', 'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
       'Feat_ATR_14', 'EMA', 'KeUp', 'KeLo', 'Feat_EMA_c', 'Feat_KeUp_c',
       'Feat_KeLo_c', 'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 'Feat_EMA_h',
       'Feat_KeUp_h', 'Feat_KeLo_h', 'Feat_EMA_l', 'Feat_KeUp_l',
       'Feat_KeLo_l', 'Feat_gains

In [67]:
X = []
Y = [] 
feature_col = [
#     'open', 'high', 'low', 'close',
#                'RSI_14', 'ATR_14',
#     'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
#     'MACD', 'SIGNAL_MACD', 'HIST',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
    'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces = []
for i in range(4,len(df_an)):
#     if df_an.SIGNAL[i] != NONE:
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [15]:
print(len(X))
print(len(Y))
print(len(indeces))

8644
8644
8644


In [16]:
print(len(feature))

293


In [68]:
SPLIT_INDEX = int(0.8*len(X))

In [69]:

X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [70]:
print(len(X_train))
print(len(Y_train))
print(len(indeces_train))
print(len(X_test))
print(len(Y_test))
print(len(indeces_test))

6915
6915
6915
1729
1729
1729


In [20]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [21]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity

In [22]:
clf = MLPClassifier( solver='adam', alpha=1e-4,learning_rate_init=1e-4,
                     tol=0.0000010,
                    hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,
                    verbose=True,warm_start = True,
                    n_iter_no_change=40)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [23]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 3.80091294
Iteration 2, loss = 1.56502652
Iteration 3, loss = 1.20298295
Iteration 4, loss = 1.06131933
Iteration 5, loss = 0.99939963
Iteration 6, loss = 0.95437373
Iteration 7, loss = 0.92453024
Iteration 8, loss = 0.90920498
Iteration 9, loss = 0.89424099
Iteration 10, loss = 0.87485590
Iteration 11, loss = 0.86644447
Iteration 12, loss = 0.85806190
Iteration 13, loss = 0.83824385
Iteration 14, loss = 0.82699931
Iteration 15, loss = 0.81807347
Iteration 16, loss = 0.81315689
Iteration 17, loss = 0.81209667
Iteration 18, loss = 0.79773261
Iteration 19, loss = 0.80466293
Iteration 20, loss = 0.79009246
Iteration 21, loss = 0.77990220
Iteration 22, loss = 0.77868648
Iteration 23, loss = 0.78047600
Iteration 24, loss = 0.77145300
Iteration 25, loss = 0.75912306
Iteration 26, loss = 0.76168357
Iteration 27, loss = 0.75397275
Iteration 28, loss = 0.75543871
Iteration 29, loss = 0.74741599
Iteration 30, loss = 0.73986163
Iteration 31, loss = 0.73657957
Iteration 32, los

Iteration 253, loss = 0.34001616
Iteration 254, loss = 0.34096759
Iteration 255, loss = 0.33323493
Iteration 256, loss = 0.34413006
Iteration 257, loss = 0.34957226
Iteration 258, loss = 0.33463745
Iteration 259, loss = 0.33691317
Iteration 260, loss = 0.33232330
Iteration 261, loss = 0.33561221
Iteration 262, loss = 0.32571220
Iteration 263, loss = 0.33050737
Iteration 264, loss = 0.32387563
Iteration 265, loss = 0.33151261
Iteration 266, loss = 0.32495283
Iteration 267, loss = 0.32282038
Iteration 268, loss = 0.32417783
Iteration 269, loss = 0.32219688
Iteration 270, loss = 0.31720353
Iteration 271, loss = 0.31812808
Iteration 272, loss = 0.32435280
Iteration 273, loss = 0.32385517
Iteration 274, loss = 0.31671124
Iteration 275, loss = 0.31795054
Iteration 276, loss = 0.32885236
Iteration 277, loss = 0.31146749
Iteration 278, loss = 0.31735768
Iteration 279, loss = 0.31138185
Iteration 280, loss = 0.30565549
Iteration 281, loss = 0.30585122
Iteration 282, loss = 0.30886299
Iteration 

Iteration 502, loss = 0.12985221
Iteration 503, loss = 0.13339559
Iteration 504, loss = 0.13144416
Iteration 505, loss = 0.13314964
Iteration 506, loss = 0.12582100
Iteration 507, loss = 0.12718776
Iteration 508, loss = 0.12645671
Iteration 509, loss = 0.12685708
Iteration 510, loss = 0.12289502
Iteration 511, loss = 0.12891784
Iteration 512, loss = 0.13196362
Iteration 513, loss = 0.12382850
Iteration 514, loss = 0.12051719
Iteration 515, loss = 0.11994624
Iteration 516, loss = 0.11949783
Iteration 517, loss = 0.12260877
Iteration 518, loss = 0.12049382
Iteration 519, loss = 0.11739356
Iteration 520, loss = 0.12028388
Iteration 521, loss = 0.11987407
Iteration 522, loss = 0.12184275
Iteration 523, loss = 0.11767860
Iteration 524, loss = 0.12107634
Iteration 525, loss = 0.11886711
Iteration 526, loss = 0.11471655
Iteration 527, loss = 0.11767477
Iteration 528, loss = 0.11441587
Iteration 529, loss = 0.11563559
Iteration 530, loss = 0.11331629
Iteration 531, loss = 0.11512707
Iteration 

Iteration 751, loss = 0.03545633
Iteration 752, loss = 0.03554802
Iteration 753, loss = 0.03493878
Iteration 754, loss = 0.03373844
Iteration 755, loss = 0.03665709
Iteration 756, loss = 0.03625647
Iteration 757, loss = 0.03465767
Iteration 758, loss = 0.03561884
Iteration 759, loss = 0.03349471
Iteration 760, loss = 0.03288349
Iteration 761, loss = 0.03333218
Iteration 762, loss = 0.03283498
Iteration 763, loss = 0.03232802
Iteration 764, loss = 0.03246142
Iteration 765, loss = 0.03330682
Iteration 766, loss = 0.03267625
Iteration 767, loss = 0.03337640
Iteration 768, loss = 0.03246166
Iteration 769, loss = 0.03396569
Iteration 770, loss = 0.03274967
Iteration 771, loss = 0.03193909
Iteration 772, loss = 0.03127219
Iteration 773, loss = 0.03239551
Iteration 774, loss = 0.03629930
Iteration 775, loss = 0.03235363
Iteration 776, loss = 0.02942365
Iteration 777, loss = 0.03021776
Iteration 778, loss = 0.02992420
Iteration 779, loss = 0.03080040
Iteration 780, loss = 0.03208722
Iteration 

Iteration 1000, loss = 0.00649070
Iteration 1001, loss = 0.00634803
Iteration 1002, loss = 0.00648939
Iteration 1003, loss = 0.00641728
Iteration 1004, loss = 0.00651530
Iteration 1005, loss = 0.00639883
Iteration 1006, loss = 0.00647840
Iteration 1007, loss = 0.00668665
Iteration 1008, loss = 0.00694220
Iteration 1009, loss = 0.00668248
Iteration 1010, loss = 0.00706721
Iteration 1011, loss = 0.00636442
Iteration 1012, loss = 0.00618708
Iteration 1013, loss = 0.00611746
Iteration 1014, loss = 0.00635255
Iteration 1015, loss = 0.00669633
Iteration 1016, loss = 0.00614421
Iteration 1017, loss = 0.00622845
Iteration 1018, loss = 0.00630759
Iteration 1019, loss = 0.00648908
Iteration 1020, loss = 0.00617668
Iteration 1021, loss = 0.00599539
Iteration 1022, loss = 0.00611997
Iteration 1023, loss = 0.00614663
Iteration 1024, loss = 0.00602682
Iteration 1025, loss = 0.00599775
Iteration 1026, loss = 0.00572965
Iteration 1027, loss = 0.00594869
Iteration 1028, loss = 0.00618409
Iteration 1029

Iteration 1241, loss = 0.00264662
Iteration 1242, loss = 0.00267751
Iteration 1243, loss = 0.00259944
Iteration 1244, loss = 0.00265106
Iteration 1245, loss = 0.00267381
Iteration 1246, loss = 0.00254310
Iteration 1247, loss = 0.00264700
Iteration 1248, loss = 0.00253274
Iteration 1249, loss = 0.00248559
Iteration 1250, loss = 0.00273644
Iteration 1251, loss = 0.00272724
Iteration 1252, loss = 0.00255544
Iteration 1253, loss = 0.00255404
Iteration 1254, loss = 0.00243306
Iteration 1255, loss = 0.00264752
Iteration 1256, loss = 0.00245267
Iteration 1257, loss = 0.00264182
Iteration 1258, loss = 0.00258294
Iteration 1259, loss = 0.00301996
Iteration 1260, loss = 0.00313315
Iteration 1261, loss = 0.00248922
Iteration 1262, loss = 0.00263713
Iteration 1263, loss = 0.00243713
Iteration 1264, loss = 0.00230470
Iteration 1265, loss = 0.00240792
Iteration 1266, loss = 0.00255589
Iteration 1267, loss = 0.00237826
Iteration 1268, loss = 0.00258096
Iteration 1269, loss = 0.00353747
Iteration 1270

Iteration 1482, loss = 0.00147084
Iteration 1483, loss = 0.00146644
Iteration 1484, loss = 0.00145952
Iteration 1485, loss = 0.00146088
Iteration 1486, loss = 0.00145752
Iteration 1487, loss = 0.00145153
Iteration 1488, loss = 0.00145329
Iteration 1489, loss = 0.00145058
Iteration 1490, loss = 0.00145140
Iteration 1491, loss = 0.00145958
Iteration 1492, loss = 0.00144617
Iteration 1493, loss = 0.00144590
Iteration 1494, loss = 0.00143474
Iteration 1495, loss = 0.00143265
Iteration 1496, loss = 0.00143455
Iteration 1497, loss = 0.00143420
Iteration 1498, loss = 0.00143169
Iteration 1499, loss = 0.00142462
Iteration 1500, loss = 0.00142480
Iteration 1501, loss = 0.00142212
Iteration 1502, loss = 0.00141773
Iteration 1503, loss = 0.00141341
Iteration 1504, loss = 0.00142643
Iteration 1505, loss = 0.00141344
Iteration 1506, loss = 0.00141816
Iteration 1507, loss = 0.00140655
Iteration 1508, loss = 0.00140306
Iteration 1509, loss = 0.00140616
Iteration 1510, loss = 0.00140622
Iteration 1511

MLPClassifier(hidden_layer_sizes=(200, 200), learning_rate_init=0.0001,
              max_iter=100000, n_iter_no_change=40, random_state=1, tol=1e-06,
              verbose=True, warm_start=True)

In [95]:
len(clf.coefs_)

3

In [99]:
len(clf.intercepts_)

3

In [103]:
clf.intercepts_[2].shape

(3,)

In [98]:
clf.coefs_[2].shape
 

(200, 3)

In [113]:
x = np.array(X_test[0]).reshape(1,-1)

In [135]:


def softmax(x):
#     return(np.exp(x)/np.exp(x).sum())
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())


In [143]:
l1 = np.matmul(np.array(X_test),clf.coefs_[0])+clf.intercepts_[0]
print(l1.shape)
l2 = np.matmul(l1,clf.coefs_[1])+clf.intercepts_[1]
print(l2.shape)
l3 = np.matmul(l2,clf.coefs_[2])+clf.intercepts_[2]
l3.shape

(1729, 200)
(1729, 200)


(1729, 3)

In [147]:
clf.out_activation_

'softmax'

In [144]:
pred = np.array([softmax(x) for x in l3])

In [146]:
pred[-1]

array([2.52966368e-278, 0.00000000e+000, 1.00000000e+000])

In [141]:
y_perdict

array([ 0,  1,  0, ...,  0, -1, -1], dtype=int64)

In [139]:
y_perdict = clf.predict(X_test)

In [140]:
clf.score(X_train, Y_train)

1.0

In [26]:
clf.score(X_test, Y_test)

0.5245806824754193

In [37]:
conf = confusion_matrix(Y_test, y_perdict)

In [28]:
pickle.dump(clf, open(f"{pair}.pickle", "wb"))

In [29]:
y_prob = clf.predict_proba(X_test)

In [30]:
for i in range(len(Y_test)):
    if Y_test[i] != y_perdict[i]:
        print(Y_test[i])
        print(y_perdict[i])
        print(y_prob[i])

-1
1
[0.00152531 0.01068404 0.98779065]
-1
0
[1.40551565e-08 9.99999986e-01 9.53126752e-25]
-1
0
[1.29662570e-02 9.87033743e-01 8.68870518e-17]
0
-1
[9.99998262e-01 1.73824038e-06 1.13784881e-17]
0
1
[4.09002450e-10 1.56471238e-03 9.98435287e-01]
0
1
[2.89165110e-11 4.24978548e-06 9.99995750e-01]
0
-1
[9.50039352e-01 4.99606480e-02 2.45007691e-12]
0
-1
[9.69811395e-01 3.01886004e-02 4.97709496e-09]
0
-1
[1.00000000e+00 7.19234322e-17 1.41183531e-32]
-1
0
[9.07039031e-04 9.99092364e-01 5.96963671e-07]
-1
0
[5.06098566e-03 9.94939014e-01 2.53371168e-18]
1
0
[1.39440193e-13 9.92796107e-01 7.20389280e-03]
0
-1
[9.99939258e-01 6.07418467e-05 2.62842489e-12]
0
-1
[9.99999997e-01 2.95640565e-09 1.21798757e-21]
0
1
[0.34895195 0.29763507 0.35341298]
0
-1
[1.00000000e+00 2.81670958e-17 4.27341898e-18]
0
1
[4.87114371e-09 1.68690500e-03 9.98313090e-01]
0
1
[9.35903516e-10 3.70834591e-07 9.99999628e-01]
0
-1
[9.97583372e-01 2.41662790e-03 1.00525529e-19]
1
0
[3.36072931e-10 9.99978141e-01 2.18588

In [40]:
conf[0,0]/(conf[0,:].sum())

0.4269340974212034

In [50]:
conf[1,1]/(conf[1,:].sum())

0.5881792183031459

In [51]:
conf[2,2]/(conf[2,:].sum())

0.4259818731117825

In [33]:
Y_test = np.array(Y_test)

In [34]:
(Y_test==-1).sum()

349

In [36]:
149+186+14

349

In [ ]:
indeces_test[0]

In [81]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
        if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_13672\4200224076.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [82]:
df_test

,time,open,high,low,close,TP,SL,pivots_l,pivots_h,SIGNAL
0,2022-08-02 08:00:00,1.22479,1.22551,1.22384,1.22435,0.0,0.0,0.0,0.0,0
1,2022-08-02 09:00:00,1.22435,1.22435,1.22181,1.22198,0.0,0.0,0.0,0.0,0
2,2022-08-02 10:00:00,1.22198,1.22218,1.21884,1.22006,0.0,0.0,0.0,0.0,0
3,2022-08-02 11:00:00,1.22006,1.22054,1.21849,1.21906,0.0,0.0,1.0,0.0,0
4,2022-08-02 12:00:00,1.21906,1.22205,1.21902,1.22173,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
3963,2023-03-23 13:00:00,1.22886,1.23070,1.22822,1.23026,0.0,0.0,0.0,0.0,0
3964,2023-03-23 14:00:00,1.23026,1.23375,1.22650,1.22752,0.0,0.0,1.0,1.0,0
3965,2023-03-23 15:00:00,1.22753,1.23184,1.22732,1.23148,0.0,0.0,0.0,0.0,-1
3966,2023-03-23 16:00:00,1.23148,1.23187,1.22950,1.23144,0.0,0.0,0.0,0.0,0


In [83]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51584 entries, 0 to 51583
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    51584 non-null  datetime64[ns]
 1   open    51584 non-null  float64       
 2   high    51584 non-null  float64       
 3   low     51584 non-null  float64       
 4   close   51584 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.0 MB


In [84]:
SPREAD = rates.get_spread()
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.00011

In [85]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [86]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.6131284916201117
 2.0    439
-1.0    277
Name: result, dtype: int64


In [87]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

1.199999999999677
0.0007223654183973271
2.0289389067522428
0.00033764864775836004
1.8749999999998843
1.943319838056493
4.139107111804193e-05
0.00015729778955204276
0.00027256058278403406
1.8454106280192775
8.18384182272252e-05
1.9142857142855754
0.00015543575185920344
0.00013906840529441883
2.0375586854460477
1.8550724637684237
1.81395348837094
2.1988950276243266
0.0002802898526829567
2.008426966292253
3.514285714286445
0.0003132599645522006
4.121264074107208e-05
6.599407703165241e-05
2.0053763440860566
0.00024061797333280904
2.2499999999999183
2.009237875288693
2.0896860986544925
0.0010690755397174106
2.1395348837209824
2.0482573726541142
0.00036037244072705595
8.382721534428266e-06
0.0007258609047939406
0.0002114075514778094
0.00027078943582702975
0.23529411764728933
5.922867344676395e-05
2.2386831275720334
2.5490695895976286e-05
0.00020388225799583394
1.9307692307689521
3.5215517241376886
0.0060790273556628974
2.0707964601772675
2.132000000000068
4.2397673215783393e-05
0.00011847839

C:\Users\meysam\AppData\Local\Temp\ipykernel_13672\1282963530.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [88]:
min(df_res_m5['balance'])

87.88313740457741

In [92]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])

In [ ]:
#  cross test
pair_cross = "EURUSD"
NB_H1_CANDLES_CROSS = 2000
rates = Rates(pair_cross, NB_H1_CANDLES_CROSS, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an_cross = rates.get_rates_from_now()
df_an_cross.drop(NB_H1_CANDLES_CROSS-1,inplace=True) 
df_an_cross.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an_cross = apply_patterns(df_an_cross)
for cp in candle_patterns:
    df_an_cross[cp] = df_an_cross[cp].astype(float)

df_an_cross = RSI(df_an_cross)
df_an_cross = BollingerBands(df_an_cross)
df_an_cross = ATR(df_an_cross)
df_an_cross = KeltnerChannels(df_an_cross)
df_an_cross = MACD(df_an_cross)

df_an_cross['pivots_l'] = False
df_an_cross['pivots_h'] = False

df_an_cross['low_perv'] = df_an_cross.low.shift(1)
df_an_cross['high_perv'] = df_an_cross.high.shift(1)
df_an_cross['low_next'] = df_an_cross.low.shift(-1)
df_an_cross['high_next'] = df_an_cross.high.shift(-1)
df_an_cross['pivots_l'] = (df_an_cross.low_perv > df_an_cross.low) & (df_an_cross.low_next > df_an_cross.low)
df_an_cross['pivots_h'] = (df_an_cross.high_perv < df_an_cross.high) & (df_an_cross.high_next < df_an_cross.high)
df_an_cross.dropna(inplace=True)
df_an_cross.reset_index(drop=True, inplace=True)

df_an_cross.head()

0.00011

In [ ]:
BUY = 1
SELL = -1
NONE = 0

df_an_cross['SIGNAL'] = 0
df_an_cross['TP'] = 0
df_an_cross['SL'] = 0

for i in range(1,len(df_an_cross)):
    if df_an_cross.pivots_l[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.low[j] <= df_an_cross.low[i-1] ):
                break
            else:
                if(df_an_cross.high[j] > 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = BUY
                    df_an_cross['SL'][i] = df_an_cross.low[i-1] #- SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i] #+ SPREAD
                    break
    if df_an_cross.pivots_h[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.high[j] >= df_an_cross.high[i-1] ):
                break
            else:
                if(df_an_cross.low[j] < 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = SELL
                    df_an_cross['SL'][i] = df_an_cross.high[i-1] #+ SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]
                    break

In [ ]:
df_an_cross.pivots_l = df_an_cross.pivots_l.astype(float)
df_an_cross.pivots_h = df_an_cross.pivots_h.astype(float)

In [ ]:
X_cross = []
Y_cross = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces_cross = []
for i in range(4,len(df_an_cross)):
    if df_an_cross.pivots_h[i-1] or df_an_cross.pivots_l[i-1]:
#         print("im here")
        feature = []
        feature.append(df_an_cross.pivots_h[i-1])
        feature.append(df_an_cross.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an_cross[fc][i])
            feature.append(df_an_cross[fc][i-1])
            feature.append(df_an_cross[fc][i-2])
            feature.append(df_an_cross[fc][i-3])
            feature.append(df_an_cross[fc][i-4])

        X_cross.append(feature)
        Y_cross.append(df_an_cross.SIGNAL[i])
        indeces_cross.append(i)

In [ ]:
len(feature)

In [ ]:
# clf = pickle.load(open("GBPUSD.pickle", "rb"))

In [ ]:
y_perdict_cross = clf.predict(X_cross)

In [ ]:
confusion_matrix(Y_cross, y_perdict_cross)

In [ ]:
(Y_cross==y_perdict_cross).sum()/len(y_perdict_cross)

In [ ]:
df_test = df_an_cross[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']]#.iloc[indeces_cross[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_cross):
    if i > 0:
        if df_test.pivots_l[idx-1] and y_perdict_cross[i] == BUY:
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]
        if df_test.pivots_h[idx-1] and y_perdict_cross[i] == SELL:
#         if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]

df_test.reset_index(drop=True, inplace=True)


In [ ]:
rates = Rates(pair_cross, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

0.0001100000000000545

In [ ]:
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])